## Q4) & Q5) Conclude the analysis by fitting a new model on these selectedvariables and reportthe same. Report the accuracy

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE

%matplotlib notebook

### Removing unncessary columns 
**TAIL_NUM = No Uniuqe information is encoded in the TAIL_NUM, the FL_Num fullfill that gap.** 

**FL_DATE = No Uniuqe information is encoded in the FL_DATE, the DAY_OF_MONTH fullfill that gap.**

In [2]:
dataset=pd.read_csv('FlightDelays.csv')
LE = LabelEncoder()
dataset = dataset.drop(['FL_DATE','TAIL_NUM'],axis = 1)
dataset.head()

,CRS_DEP_TIME,CARRIER,DEP_TIME,DEST,DISTANCE,FL_NUM,ORIGIN,Weather,DAY_WEEK,DAY_OF_MONTH,Flight Status
0,1455,OH,1455,JFK,184,5935,BWI,0,4,1,ontime
1,1640,DH,1640,JFK,213,6155,DCA,0,4,1,ontime
2,1245,DH,1245,LGA,229,7208,IAD,0,4,1,ontime
3,1715,DH,1709,LGA,229,7215,IAD,0,4,1,ontime
4,1039,DH,1035,LGA,229,7792,IAD,0,4,1,ontime


In [3]:
# carrier=pd.get_dummies(dataset['CARRIER'],drop_first=True)
# destination=pd.get_dummies(dataset['DEST'],drop_first=True)
# origin=pd.get_dummies(dataset['ORIGIN'],drop_first=True)
# Flight_status=pd.get_dummies(dataset['Flight Status'],drop_first=True)

# Finaldataset = dataset.drop(['CARRIER','DEST','ORIGIN','Flight Status'],axis=1)
# Finaldataset = pd.concat([Finaldataset,carrier,destination,origin,Flight_status],axis=1)
Finaldataset = pd.get_dummies(dataset, columns = ['CARRIER', 'DEST','ORIGIN'], prefix=['CARRIER', 'DEST','ORIGIN'])

Finaldataset.head()

,CRS_DEP_TIME,DEP_TIME,DISTANCE,FL_NUM,Weather,DAY_WEEK,DAY_OF_MONTH,Flight Status,CARRIER_CO,CARRIER_DH,...,CARRIER_OH,CARRIER_RU,CARRIER_UA,CARRIER_US,DEST_EWR,DEST_JFK,DEST_LGA,ORIGIN_BWI,ORIGIN_DCA,ORIGIN_IAD
0,1455,1455,184,5935,0,4,1,ontime,0,0,...,1,0,0,0,0,1,0,1,0,0
1,1640,1640,213,6155,0,4,1,ontime,0,1,...,0,0,0,0,0,1,0,0,1,0
2,1245,1245,229,7208,0,4,1,ontime,0,1,...,0,0,0,0,0,0,1,0,0,1
3,1715,1709,229,7215,0,4,1,ontime,0,1,...,0,0,0,0,0,0,1,0,0,1
4,1039,1035,229,7792,0,4,1,ontime,0,1,...,0,0,0,0,0,0,1,0,0,1


In [4]:
Finaldataset.drop(['Flight Status'],axis =1).columns

Index(['CRS_DEP_TIME', 'DEP_TIME', 'DISTANCE', 'FL_NUM', 'Weather', 'DAY_WEEK',
       'DAY_OF_MONTH', 'CARRIER_CO', 'CARRIER_DH', 'CARRIER_DL', 'CARRIER_MQ',
       'CARRIER_OH', 'CARRIER_RU', 'CARRIER_UA', 'CARRIER_US', 'DEST_EWR',
       'DEST_JFK', 'DEST_LGA', 'ORIGIN_BWI', 'ORIGIN_DCA', 'ORIGIN_IAD'],
      dtype='object')

In [5]:
x_total_df = Finaldataset.drop(['Flight Status'],axis =1)
x_total = x_total_df.values
le = LabelEncoder()
Finaldataset['Flight Status'] = le.fit_transform(Finaldataset['Flight Status'])
y = Finaldataset['Flight Status'].values
y

array([1, 1, 1, ..., 1, 1, 1])

In [6]:
xtrain,xtest,ytrain,ytest = train_test_split(x_total,y,test_size=0.4,random_state=42)

In [7]:
def Logistic_Regression_lasso(xtrain,xtest,ytrain,ytest):
    model = LogisticRegression(solver='liblinear',random_state=0, max_iter = 100000,penalty='l1')
    model.fit(xtrain,ytrain)
    predictions = model.predict(xtest)
    accuracy = confusion_matrix(ytest,predictions)
    print(model.coef_ )
    print(accuracy)
    print(accuracy_score(ytest,predictions))
    return model

In [8]:
model = Logistic_Regression_lasso(xtrain,xtest,ytrain,ytest)

[[ 2.21659094e-02 -2.27701564e-02  1.27561426e-02  6.18001325e-05
  -3.19810710e+00  5.58779260e-02 -2.11911830e-02 -5.60020602e-01
   0.00000000e+00  2.83185087e-01 -9.45181355e-01  5.30891732e-01
  -3.25219656e-01  8.40506137e-02  7.64279792e-02  0.00000000e+00
   9.10121246e-02 -7.61381508e-02  2.83171963e-01  0.00000000e+00
  -5.99080266e-01]]
[[ 73  90]
 [  1 717]]
0.8967082860385925


In [9]:
order = model.coef_[0]
featurelist = {}
for weight,col in zip(order,x_total_df.columns):
    featurelist.update({col :[col,weight]})
# dict(sorted(featurelist.items(),key=lambda x: x[1],reverse=True))

pd.DataFrame(pd.DataFrame(featurelist.values(),columns=["Feature", "Regularized Weight"]))

,Feature,Regularized Weight
0,CRS_DEP_TIME,0.022166
1,DEP_TIME,-0.022770
2,DISTANCE,0.012756
3,FL_NUM,0.000062
4,Weather,-3.198107
5,DAY_WEEK,0.055878
6,DAY_OF_MONTH,-0.021191
7,CARRIER_CO,-0.560021
8,CARRIER_DH,0.000000
9,CARRIER_DL,0.283185


In [10]:
def Logistic_Regression(xtrain,xtest,ytrain,ytest):
    model = LogisticRegression(random_state=0, max_iter = 1000)
    model.fit(xtrain,ytrain)
    predictions = model.predict(xtest)
    accuracy = confusion_matrix(ytest,predictions)
    print(model.coef_ )
    print(model.intercept_)
    print(confusion_matrix(predictions,ytest))    
    print(accuracy_score(ytest,predictions))
    return model.coef_[0] 

In [14]:
x_selected = Finaldataset.drop(['ORIGIN_DCA','FL_NUM','DEST_EWR','CARRIER_DH','Flight Status'],axis =1).values
x_train,x_test,y_train,y_test = train_test_split(x_selected,y,test_size=0.4,random_state=42)
Selected_model = Logistic_Regression(x_train,x_test,y_train,y_test)
# x_selected.head()

[[ 0.02242457 -0.02301188  0.01315855 -2.24207551  0.06010609 -0.0224782
  -0.79044658  0.28143692 -0.79208235  0.69897291 -0.59636474  0.11081238
   0.16179945  0.05440061 -0.22548781  0.57249364 -0.37733514]]
[0.15266861]
[[ 73   1]
 [ 90 717]]
0.8967082860385925


## Q6) Find the ideal weather conditions for the highest chance of an ontime flightfrom DC to New York . (weather, time, day, carrier)


**ANS = In the Report (according of analysis of the  barplot and  boxplots) \
The best flights  from DC to NY are shown below**

In [15]:
dataset[(dataset['Weather'] == 0) & (dataset['DAY_WEEK'] == 6) & (dataset['CARRIER'] == 'US') & (dataset['CRS_DEP_TIME'] == 1300)]

,CRS_DEP_TIME,CARRIER,DEP_TIME,DEST,DISTANCE,FL_NUM,ORIGIN,Weather,DAY_WEEK,DAY_OF_MONTH,Flight Status
141,1300,US,1256,LGA,214,2172,DCA,0,6,3,ontime
670,1300,US,1256,LGA,214,2172,DCA,0,6,10,ontime
1189,1300,US,1259,LGA,214,2172,DCA,0,6,17,ontime
1712,1300,US,1259,LGA,214,2172,DCA,0,6,24,ontime
2180,1300,US,1256,LGA,214,2172,DCA,0,6,31,ontime
